<a href="https://colab.research.google.com/github/ralsouza/apache_spark_real_time_analytics/blob/master/notebooks/11_pyspark_mllib_clustering_k_means.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Clustering K-Means

**Description:**
*   Unsupervised algorithm;
*   Grouping data by simility;
*   Partitions the data by a `k` number of clusters, with each observation belongs to a just one cluster;
*   The clusterizations is made measuring the distance between the data points and grouping them;
*   Multiple distance measures may be used, such as, `Euclidian Distance` and `Manhattan Distance`;

**Advantages:**
*   Faster;
*   Efficient to many variables;

**Disvantages:**
*   The `k` value must be know;
*   The initial value of `k` influences on created clusters;

**Application:**
*   Priliminary grouping before applying classification techniques;
*   Geographic clustering;



# Setup Spark


In [ ]:
!apt-get update

In [2]:
# Install the dependencies
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-2.4.4/spark-2.4.4-bin-hadoop2.7.tgz
!tar xf spark-2.4.4-bin-hadoop2.7.tgz
!pip install -q findspark

In [3]:
# Environment variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.4-bin-hadoop2.7"

In [4]:
# Make pyspark "importable"
import findspark
findspark.init('spark-2.4.4-bin-hadoop2.7')

In [5]:
# Libraries and Context Setup
import pyspark
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf

In [6]:
# create the session
conf = SparkConf().set("spark.ui.port", "4050")

# create the context
sc = pyspark.SparkContext(conf=conf)


# Instance Spark Session
spark = SparkSession.builder.master('local').appName('spark_ml_lib').getOrCreate()

# Create the SQL Context
sqlContext = pyspark.SQLContext(sc)

# Grouping cars

In [10]:
# Libraries
import math
import pandas as pd
from pyspark.sql import Row
from pyspark.ml.linalg import Vectors
from pyspark.ml.clustering import KMeans
import matplotlib.pylab as plt
%matplotlib inline